# U-Net

In [2]:
import timm
print(timm.list_models("*u*"))

['cs3darknet_focus_l', 'cs3darknet_focus_m', 'cs3darknet_focus_s', 'cs3darknet_focus_x', 'deit3_huge_patch14_224', 'deit3_huge_patch14_224_in21ft1k', 'densenetblur121d', 'ecaresnet50d_pruned', 'ecaresnet101d_pruned', 'efficientnet_b1_pruned', 'efficientnet_b2_pruned', 'efficientnet_b3_pruned', 'efficientnet_el_pruned', 'efficientnet_es_pruned', 'gluon_inception_v3', 'gluon_resnet18_v1b', 'gluon_resnet34_v1b', 'gluon_resnet50_v1b', 'gluon_resnet50_v1c', 'gluon_resnet50_v1d', 'gluon_resnet50_v1s', 'gluon_resnet101_v1b', 'gluon_resnet101_v1c', 'gluon_resnet101_v1d', 'gluon_resnet101_v1s', 'gluon_resnet152_v1b', 'gluon_resnet152_v1c', 'gluon_resnet152_v1d', 'gluon_resnet152_v1s', 'gluon_resnext50_32x4d', 'gluon_resnext101_32x4d', 'gluon_resnext101_64x4d', 'gluon_senet154', 'gluon_seresnext50_32x4d', 'gluon_seresnext101_32x4d', 'gluon_seresnext101_64x4d', 'gluon_xception65', 'resnetblur18', 'resnetblur50', 'resnetblur50d', 'resnetblur101d', 'sequencer2d_l', 'sequencer2d_m', 'sequencer2d_s',